## Preparing DT-Climate date

This notebook prepares DT-climate data by downloading it through polytope, interpolating it to a regular lat-lon grid and selecting the area of interest. This notebook assumes prior authentication using the desp-authentication.py script.

In [18]:
import earthkit.data
import earthkit.plots
import earthkit.regrid
import hydromt # for setting crs to raster

These are the bounds of the area of interest in [W, S, E, N]. Also define the filename for the final netcdf data file

In [ ]:
bounds = [-15.0, 43.0, 13.0, 63.0]
filename = ""

Details of the download request. Check the DT-Climate documentation for allowed values

In [5]:
request = {
    "activity": "ScenarioMIP",
    "class": "d1",
    "dataset": "climate-dt",
    "date": "20241102/to/20241104",
    "experiment": "SSP3-7.0",
    "expver": "0001",
    "generation": "1",
    "levtype": "sfc",
    "model": "IFS-NEMO",
    "param": "134/165/166",
    "realization": "1",
    "resolution": "standard",
    "stream": "clte",
    "time": '0000/0100/0200/0300/0400/0500/0600/0700/0800/0900/1000/1100/1200/1300/1400/1500/1600/1700/1800/1900/2000/2100/2200/2300',
    "type": "fc"
}

Download data into memory

In [6]:
data = earthkit.data.from_source(
    "polytope",
    "destination-earth",
    request,
    address="polytope.lumi.apps.dte.destination-earth.eu",
    stream=False
)

2025-04-02 10:46:46 - INFO - Key read from C:\Users\tromp_wm\.polytopeapirc
2025-04-02 10:46:46 - INFO - Sending request...
{'request': 'activity: ScenarioMIP\n'
            'class: d1\n'
            'dataset: climate-dt\n'
            'date: 20241102/to/20241104\n'
            'experiment: SSP3-7.0\n'
            "expver: '0001'\n"
            "generation: '1'\n"
            'levtype: sfc\n'
            'model: IFS-NEMO\n'
            'param: 134/165/166\n'
            "realization: '1'\n"
            'resolution: standard\n'
            'stream: clte\n'
            'time: '
            '0000/0100/0200/0300/0400/0500/0600/0700/0800/0900/1000/1100/1200/1300/1400/1500/1600/1700/1800/1900/2000/2100/2200/2300\n'
            'type: fc\n',
 'verb': 'retrieve'}
2025-04-02 10:46:46 - INFO - Polytope user key found in session cache for user tromp_wm


2025-04-02 10:46:46 - INFO - Request accepted. Please poll ./8d5669d8-ad58-42b2-b8ac-63db4df72a0d for status
2025-04-02 10:46:46 - INFO - Polytope user key found in session cache for user tromp_wm
2025-04-02 10:46:46 - INFO - Checking request status (./8d5669d8-ad58-42b2-b8ac-63db4df72a0d)...
2025-04-02 10:46:46 - INFO - The current status of the request is 'processing'
2025-04-02 10:46:51 - INFO - The current status of the request is 'processed'


8d5669d8-ad58-42b2-b8ac-63db4df72a0d.grib:   0%|          | 0.00/98.8M [00:00<?, ?B/s]

In [7]:
data.ls()

,centre,shortName,typeOfLevel,level,dataDate,dataTime,stepRange,dataType,number,gridType
0,ecmf,sp,surface,0,20241102,0,0,fc,None,healpix
1,ecmf,10u,heightAboveGround,10,20241102,0,0,fc,None,healpix
2,ecmf,10v,heightAboveGround,10,20241102,0,0,fc,None,healpix
3,ecmf,sp,surface,0,20241102,100,0,fc,None,healpix
4,ecmf,10u,heightAboveGround,10,20241102,100,0,fc,None,healpix
...,...,...,...,...,...,...,...,...,...,...
211,ecmf,10u,heightAboveGround,10,20241104,2200,0,fc,None,healpix
212,ecmf,10v,heightAboveGround,10,20241104,2200,0,fc,None,healpix
213,ecmf,sp,surface,0,20241104,2300,0,fc,None,healpix
214,ecmf,10u,heightAboveGround,10,20241104,2300,0,fc,None,healpix


Interpolate data to a regular lat-lon grid of 0.1deg resolution

In [8]:
data_latlon = earthkit.regrid.interpolate(data, out_grid={"grid": [0.1, 0.1]}, method="linear")

ce05761e06ddc639c16cad8e47ca64ded634218b6c6bca2bc72c14e472d38825.npz:   0%|          | 0.00/84.5M [00:00<?, ?B…

Turn data into xarray object for further processing

In [41]:
ds = data_latlon.to_xarray(engine='cfgrib', xarray_open_dataset_kwargs={"chunks": {"time": 1}}).squeeze()

Shift longitude values and rename variables to names recognized by HydroMT/FloodAdapt

In [43]:
ds = ds.assign_coords(
    {"longitude": ((ds.longitude+180)%360)-180}
)

ds = ds.sortby("longitude")
ds = ds.sortby("latitude")

ds = ds.rename(
    {
        "longitude": "x",
        "latitude": "y",
        "sp": "press_msl",
        "u10": "wind10_u",
        "v10": "wind10_v"
    }
)

Define a crs for the data

In [44]:
ds.raster.set_crs(4326)

<Geographic 2D CRS: EPSG:4326>
Name: WGS 84
Axis Info [ellipsoidal]:
- Lat[north]: Geodetic latitude (degree)
- Lon[east]: Geodetic longitude (degree)
Area of Use:
- name: World.
- bounds: (-180.0, -90.0, 180.0, 90.0)
Datum: World Geodetic System 1984 ensemble
- Ellipsoid: WGS 84
- Prime Meridian: Greenwich

Select the area of interest

In [45]:
ds = ds.sel(
    x = slice(bounds[0], bounds[2]),
    y = slice(bounds[1], bounds[3])
)

Save data to netcdf

In [ ]:
ds.to_netcdf(filename)